In [1]:
from old_code.dataset import *
import tensorflow as tf
from MLP import *

2025-07-02 20:45:26.059755: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-02 20:45:26.101830: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-02 20:45:26.922184: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
%load_ext autoreload
%autoreload 2

In [3]:
# Test training LinearModel on 2024-08-21 data
import pickle
import os
import numpy as np
from old_code.dataset import get_scaled_split_data
from MLP import MLP
from Model import Model
from Model import EnsembleModel
import pandas as pd
from Dataset import Dataset


In [6]:

df=pd.DataFrame({
    'A': [1, 2, 3],
    'b': [1, 1, 3],
    'B': [1, 4, 3],
    'a': [1, 9, 3],
})
df

,A,b,B,a
0,1,1,1,1
1,2,1,4,9
2,3,3,3,3


In [11]:
!ls "../Features"

2024-04-15  2024-08-21	2025-01-04  2025-05-09	2025-05-25


In [4]:

ds = Dataset("../Features/2024-08-21", "../Features/2024-08-21")

In [43]:
type(ml[1])

str

In [72]:
ds.yCC

cell_num  cycle
1         2        0.144482
          2        0.144440
          2        0.144396
          2        0.144356
          2        0.144332
                     ...   
55        21      -0.107454
          21      -0.107569
          21      -0.107619
          21      -0.107538
          21      -0.107533
Name: NEP, Length: 9785838, dtype: float64

In [35]:
df = ds.xCC
ml = ds.masterList

In [57]:
x_tr = df_with_label.loc[pd.IndexSlice[1, 2], :]

In [82]:
by = 'c-rate'
x_tr, y_tr, x_te, y_te = ds.split(by=by, test_ids=['0.33', '0.5'])
print('x_train: ', np.unique(x_tr.index.get_level_values(by)))
print('x_test: ', np.unique(x_te.index.get_level_values(by)))

x_train:  ['0.8' '1' 'stepped']
x_test:  ['0.33' '0.5']


In [84]:
by = 'cycle'
x_tr, y_tr, x_te, y_te = ds.split(by=by, test_ids=[10, 15, 26, 40, 44])
print('x_train: ', np.unique(x_tr.index.get_level_values(by)))
print('x_test: ', np.unique(x_te.index.get_level_values(by)))


x_train:  [  2   3   4   5   6   7   8   9  11  12  13  14  16  17  18  19  20  21
  22  23  24  25  27  28  29  30  31  32  33  34  35  36  37  38  39  41
  42  43  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60
  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78
  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96
  97  98  99 100]
x_test:  [10 15 26 40 44]


In [85]:
by = 'cell_num'
x_tr, y_tr, x_te, y_te = ds.split(by=by, test_ids=[10, 15, 26, 40, 44])
print('x_train: ', np.unique(x_tr.index.get_level_values(by)))
print('x_test: ', np.unique(x_te.index.get_level_values(by)))

x_train:  [ 1  2  3  4  5  6  7  8  9 11 12 13 14 16 17 18 19 20 21 22 23 24 25 27
 28 29 30 31 32 33 34 35 36 37 38 39 41 42 43 45 46 47 48 49 50 51 52 53
 54 55]
x_test:  [10 15 26 40 44]


In [78]:
ds = Dataset("../Features/2024-08-21", "../Features/2024-08-21")

ds.xCC.loc[1]

,Voltage,Current,Throughput,Energy,Capacity,Time,Overpotential,dQ_dV,dVoltage/dt,dEnergy/dt,...,Energy-2,Capacity-2,Time-2,Overpotential-2,dQ_dV-2,dVoltage/dt-2,dEnergy/dt-2,dCapacity/dt-2,dOverpotential/dt-2,Power-2
cycle,,,,,,,,,,,,,,,,,,,,,
2,3.301722,0.00106,11.096774,17.887097,11.096774,1167.1410,0.015730,0.000000,0.000112,0.016129,...,17.854839,11.064516,1165.1409,0.015911,0.000000,0.000056,0.016129,0.000000,0.000056,0.003500
2,3.301717,0.00106,11.096774,17.903226,11.096774,1168.1410,0.015725,0.000000,-0.000005,0.016129,...,17.870968,11.096774,1166.1409,0.015618,0.000000,0.000023,0.016129,0.032258,-0.000293,0.003500
2,3.301807,0.00106,11.096774,17.919355,11.096774,1169.1410,0.015815,129.032258,0.000090,0.016129,...,17.887097,11.096774,1167.1410,0.015730,0.000000,0.000112,0.016129,0.000000,0.000112,0.003500
2,3.301815,0.00106,11.129032,17.935484,11.129032,1170.1410,0.015506,193.548387,0.000008,0.016129,...,17.903226,11.096774,1168.1410,0.015725,0.000000,-0.000005,0.016129,0.000000,-0.000005,0.003500
2,3.301893,0.00106,11.129032,17.951613,11.129032,1171.1412,0.015584,193.548387,0.000078,0.016129,...,17.919355,11.096774,1169.1410,0.015815,129.032258,0.000090,0.016129,0.000000,0.000090,0.003500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,3.596219,0.00106,18094.741935,153.096774,90.225806,9499.0352,0.050599,0.000000,0.000620,0.016129,...,153.064516,90.225806,9497.0350,0.049400,64.516129,0.000625,0.016129,0.032258,0.000625,0.003811
100,3.596782,0.00106,18094.774193,153.129032,90.258065,9500.0353,0.051162,64.516129,0.000563,0.032258,...,153.080645,90.225806,9498.0350,0.049979,0.000000,0.000579,0.016129,0.000000,0.000579,0.003811
100,3.597374,0.00106,18094.774193,153.145161,90.258065,9501.0355,0.051754,0.000000,0.000592,0.016129,...,153.096774,90.225806,9499.0352,0.050599,0.000000,0.000620,0.016129,0.000000,0.000620,0.003812


In [67]:
a = [:, :, :]

SyntaxError: invalid syntax (2597501884.py, line 1)

In [49]:
df_with_label

Voltage  Current   Throughput      Energy   Capacity  \
cell_num cycle c-rate                                                           
1        2     0.33     3.301722  0.00106    11.096774   17.887097  11.096774   
               0.33     3.301717  0.00106    11.096774   17.903226  11.096774   
               0.33     3.301807  0.00106    11.096774   17.919355  11.096774   
               0.33     3.301815  0.00106    11.129032   17.935484  11.129032   
               0.33     3.301893  0.00106    11.129032   17.951613  11.129032   
...                          ...      ...          ...         ...        ...   
55       21    stepped  3.598121  0.00159  2868.000000  128.387097  74.129032   
               stepped  3.598300  0.00159  2868.032258  128.419355  74.161290   
               stepped  3.598487  0.00159  2868.032258  128.435484  74.161290   
               stepped  3.598670  0.00159  2868.032258  128.467742  74.161290   
               stepped  3.598896  0.00159  2868.064516  128.483871  74.193548   

                             Time  Overpotential       dQ_dV  dVoltage/dt  \
cell_num cycle c-rate                                                       
1        2     0.33     1167.1410       0.015730    0.000000     0.000112   
               0.33     1168.1410       0.015725    0.000000    -0.000005   
               0.33     1169.1410       0.015815  129.032258     0.000090   
               0.33     1170.1410       0.015506  193.548387     0.000008   
               0.33     1171.1412       0.015584  193.548387     0.000078   
...                           ...            ...         ...          ...   
55       21    stepped  3962.4263       0.193683   64.516129     0.000142   
               stepped  3963.4271       0.193809  129.032258     0.000179   
               stepped  3964.4273       0.193996   64.516129     0.000187   
               stepped  3965.4272       0.194179   64.516129     0.000183   
               stepped  3966.4272       0.194316   64.516129     0.000226   

                        dEnergy/dt  ...    Energy-2  Capacity-2     Time-2  \
cell_num cycle c-rate               ...                                      
1        2     0.33       0.016129  ...   17.854839   11.064516  1165.1409   
               0.33       0.016129  ...   17.870968   11.096774  1166.1409   
               0.33       0.016129  ...   17.887097   11.096774  1167.1410   
               0.33       0.016129  ...   17.903226   11.096774  1168.1410   
               0.33       0.016129  ...   17.919355   11.096774  1169.1410   
...                            ...  ...         ...         ...        ...   
55       21    stepped    0.016129  ...  128.338710   74.096774  3960.4258   
               stepped    0.032258  ...  128.370968   74.129032  3961.4260   
               stepped    0.016129  ...  128.387097   74.129032  3962.4263   
               stepped    0.032258  ...  128.419355   74.161290  3963.4271   
               stepped    0.016129  ...  128.435484   74.161290  3964.4273   

                        Overpotential-2     dQ_dV-2  dVoltage/dt-2  \
cell_num cycle c-rate                                                
1        2     0.33            0.015911    0.000000       0.000056   
               0.33            0.015618    0.000000       0.000023   
               0.33            0.015730    0.000000       0.000112   
               0.33            0.015725    0.000000      -0.000005   
               0.33            0.015815  129.032258       0.000090   
...                                 ...         ...            ...   
55       21    stepped         0.193398   64.516129       0.000205   
               stepped         0.193541  129.032258       0.000186   
               stepped         0.193683   64.516129       0.000142   
               stepped         0.193809  129.032258       0.000179   
               stepped         0.193996   64.516129       0.000187   

                        dEnergy/dt-2  dCapa

In [46]:
train

['0.33', '0.5', '1', 'stepped']

In [37]:
cell_nums = df.index.get_level_values("cell_num")
labels = cell_nums.map(ml)
new_index = pd.MultiIndex.from_arrays(
    [
        df.index.get_level_values("cell_num"),
        df.index.get_level_values("cycle"),
        labels
    ],
    names=["cell_num", "cycle", "c-rate"]
)
df_with_label = df.copy()
df_with_label.index = new_index
df_with_label

Voltage  Current   Throughput      Energy   Capacity  \
cell_num cycle c-rate                                                           
1        2     0.33     3.301722  0.00106    11.096774   17.887097  11.096774   
               0.33     3.301717  0.00106    11.096774   17.903226  11.096774   
               0.33     3.301807  0.00106    11.096774   17.919355  11.096774   
               0.33     3.301815  0.00106    11.129032   17.935484  11.129032   
               0.33     3.301893  0.00106    11.129032   17.951613  11.129032   
...                          ...      ...          ...         ...        ...   
55       21    stepped  3.598121  0.00159  2868.000000  128.387097  74.129032   
               stepped  3.598300  0.00159  2868.032258  128.419355  74.161290   
               stepped  3.598487  0.00159  2868.032258  128.435484  74.161290   
               stepped  3.598670  0.00159  2868.032258  128.467742  74.161290   
               stepped  3.598896  0.00159  2868.064516  128.483871  74.193548   

                             Time  Overpotential       dQ_dV  dVoltage/dt  \
cell_num cycle c-rate                                                       
1        2     0.33     1167.1410       0.015730    0.000000     0.000112   
               0.33     1168.1410       0.015725    0.000000    -0.000005   
               0.33     1169.1410       0.015815  129.032258     0.000090   
               0.33     1170.1410       0.015506  193.548387     0.000008   
               0.33     1171.1412       0.015584  193.548387     0.000078   
...                           ...            ...         ...          ...   
55       21    stepped  3962.4263       0.193683   64.516129     0.000142   
               stepped  3963.4271       0.193809  129.032258     0.000179   
               stepped  3964.4273       0.193996   64.516129     0.000187   
               stepped  3965.4272       0.194179   64.516129     0.000183   
               stepped  3966.4272       0.194316   64.516129     0.000226   

                        dEnergy/dt  ...    Energy-2  Capacity-2     Time-2  \
cell_num cycle c-rate               ...                                      
1        2     0.33       0.016129  ...   17.854839   11.064516  1165.1409   
               0.33       0.016129  ...   17.870968   11.096774  1166.1409   
               0.33       0.016129  ...   17.887097   11.096774  1167.1410   
               0.33       0.016129  ...   17.903226   11.096774  1168.1410   
               0.33       0.016129  ...   17.919355   11.096774  1169.1410   
...                            ...  ...         ...         ...        ...   
55       21    stepped    0.016129  ...  128.338710   74.096774  3960.4258   
               stepped    0.032258  ...  128.370968   74.129032  3961.4260   
               stepped    0.016129  ...  128.387097   74.129032  3962.4263   
               stepped    0.032258  ...  128.419355   74.161290  3963.4271   
               stepped    0.016129  ...  128.435484   74.161290  3964.4273   

                        Overpotential-2     dQ_dV-2  dVoltage/dt-2  \
cell_num cycle c-rate                                                
1        2     0.33            0.015911    0.000000       0.000056   
               0.33            0.015618    0.000000       0.000023   
               0.33            0.015730    0.000000       0.000112   
               0.33            0.015725    0.000000      -0.000005   
               0.33            0.015815  129.032258       0.000090   
...                                 ...         ...            ...   
55       21    stepped         0.193398   64.516129       0.000205   
               stepped         0.193541  129.032258       0.000186   
               stepped         0.193683   64.516129       0.000142   
               stepped         0.193809  129.032258       0.000179   
               stepped         0.193996   64.516129       0.000187   

                        dEnergy/dt-2  dCapa

In [32]:
un = np.unique(new_index)

In [41]:
for cell_num, cycle, crate in un:
    if ml[cell_num] != crate:
        print("error")
        print(cell_num, cycle, crate)
        print()
print("sall good")

sall good


In [39]:
ml[1]

'0.33'

In [22]:
type(ml)

pandas.core.series.Series

In [44]:
from sklearn.preprocessing import RobustScaler

data1 = np.random.rand(100,1)
data2 = np.random.rand(100,1)
data1c = np.array([a for a in data1])

r1 = RobustScaler()
r2 = RobustScaler()

r1.fit(data1)
r2.fit(data2)

print(np.max(np.abs(r2.transform(data1) - r2.transform(r1.transform(data1)) )))

2.022759447906666


In [ ]:
from Dataset import Dataset

ds = Dataset('../Features/2024-08-21')


<bound method Dataset.add_shifted_series of <Dataset.Dataset object at 0x7fd38e14eee0>>

In [68]:
ds.xCC.head()

Voltage  Current  Throughput     Energy   Capacity  \
cell_num cycle                                                        
1        2      3.301722  0.00106   11.096774  17.887097  11.096774   
         2      3.301717  0.00106   11.096774  17.903226  11.096774   
         2      3.301807  0.00106   11.096774  17.919355  11.096774   
         2      3.301815  0.00106   11.129032  17.935484  11.129032   
         2      3.301893  0.00106   11.129032  17.951613  11.129032   

                     Time  Overpotential       dQ_dV  dVoltage/dt  dEnergy/dt  \
cell_num cycle                                                                  
1        2      1167.1410       0.015730    0.000000     0.000112    0.016129   
         2      1168.1410       0.015725    0.000000    -0.000005    0.016129   
         2      1169.1410       0.015815  129.032258     0.000090    0.016129   
         2      1170.1410       0.015506  193.548387     0.000008    0.016129   
         2      1171.1412       0.015584  193.548387     0.000078    0.016129   

                ...  Capacity-2     Time-2  Overpotential-2     dQ_dV-2  \
cell_num cycle  ...                                                       
1        2      ...   11.064516  1165.1409         0.015911    0.000000   
         2      ...   11.096774  1166.1409         0.015618    0.000000   
         2      ...   11.096774  1167.1410         0.015730    0.000000   
         2      ...   11.096774  1168.1410         0.015725    0.000000   
         2      ...   11.096774  1169.1410         0.015815  129.032258   

                dVoltage/dt-2  dEnergy/dt-2  dCapacity/dt-2  \
cell_num cycle                                                
1        2           0.000056      0.016129        0.000000   
         2           0.000023      0.016129        0.032258   
         2           0.000112      0.016129        0.000000   
         2          -0.000005      0.016129        0.000000   
         2           0.000090      0.016129        0.000000   

                dOverpotential/dt-2  Power-2  shifted_NEP  
cell_num cycle                                             
1        2                 0.000056   0.0035     0.000000  
         2                -0.000293   0.0035     0.144482  
         2                 0.000112   0.0035     0.144440  
         2                -0.000005   0.0035     0.144396  
         2                 0.000090   0.0035     0.144356  

[5 rows x 66 columns]

In [69]:
ds.xCC.columns

Index(['Voltage', 'Current', 'Throughput', 'Energy', 'Capacity', 'Time',
       'Overpotential', 'dQ_dV', 'dVoltage/dt', 'dEnergy/dt', 'dCapacity/dt',
       'dOverpotential/dt', 'Power', 'Voltage-8', 'Current-8', 'Throughput-8',
       'Energy-8', 'Capacity-8', 'Time-8', 'Overpotential-8', 'dQ_dV-8',
       'dVoltage/dt-8', 'dEnergy/dt-8', 'dCapacity/dt-8',
       'dOverpotential/dt-8', 'Power-8', 'Voltage-6', 'Current-6',
       'Throughput-6', 'Energy-6', 'Capacity-6', 'Time-6', 'Overpotential-6',
       'dQ_dV-6', 'dVoltage/dt-6', 'dEnergy/dt-6', 'dCapacity/dt-6',
       'dOverpotential/dt-6', 'Power-6', 'Voltage-4', 'Current-4',
       'Throughput-4', 'Energy-4', 'Capacity-4', 'Time-4', 'Overpotential-4',
       'dQ_dV-4', 'dVoltage/dt-4', 'dEnergy/dt-4', 'dCapacity/dt-4',
       'dOverpotential/dt-4', 'Power-4', 'Voltage-2', 'Current-2',
       'Throughput-2', 'Energy-2', 'Capacity-2', 'Time-2', 'Overpotential-2',
       'dQ_dV-2', 'dVoltage/dt-2', 'dEnergy/dt-2', 'dCapacity/dt-2

In [79]:
ds = Dataset('../Features/2024-08-21')
ds.subsample(int(3e6))
ds.xCC.tail()

Voltage  Current   Throughput      Energy   Capacity  \
cell_num cycle                                                          
5        26     3.539944  0.00159  4494.677419  148.129032  87.064516   
         26     3.540077  0.00159  4494.677419  148.145161  87.064516   
         26     3.540275  0.00159  4494.709677  148.177419  87.096774   
         26     3.540527  0.00159  4494.709677  148.193548  87.096774   
         26     3.540710  0.00159  4494.741935  148.225806  87.129032   

                     Time  Overpotential       dQ_dV  dVoltage/dt  dEnergy/dt  \
cell_num cycle                                                                  
5        26     6109.8185      -0.008888   64.516129     0.000282    0.032258   
         26     6110.8186      -0.008755   64.516129     0.000133    0.016129   
         26     6111.8187      -0.008557  129.032258     0.000198    0.032258   
         26     6112.8189      -0.008305   64.516129     0.000252    0.016129   
         26     6113.8190      -0.008122   64.516129     0.000183    0.032258   

                ...    Energy-2  Capacity-2     Time-2  Overpotential-2  \
cell_num cycle  ...                                                       
5        26     ...  148.080645   87.032258  6107.8183        -0.009385   
         26     ...  148.096774   87.032258  6108.8185        -0.009170   
         26     ...  148.129032   87.064516  6109.8185        -0.008888   
         26     ...  148.145161   87.064516  6110.8186        -0.008755   
         26     ...  148.177419   87.096774  6111.8187        -0.008557   

                   dQ_dV-2  dVoltage/dt-2  dEnergy/dt-2  dCapacity/dt-2  \
cell_num cycle                                                            
5        26      64.516129       0.000154      0.032258        0.032258   
         26      64.516129       0.000215      0.016129        0.000000   
         26      64.516129       0.000282      0.032258        0.032258   
         26      64.516129       0.000133      0.016129        0.000000   
         26     129.032258       0.000198      0.032258        0.032258   

                dOverpotential/dt-2   Power-2  
cell_num cycle                                 
5        26                0.000154  0.005628  
         26                0.000215  0.005628  
         26                0.000282  0.005629  
         26                0.000133  0.005629  
         26                0.000198  0.005629  

[5 rows x 65 columns]

In [64]:
new_col = "shifted"
df = ds.xCC
series_data = ds.yCC        
if new_col is None:
    base = series_data.name or "series"
    new_col = f"shifted_{base}"

out = df.copy()
out[new_col] = 0.0

cell_idx = df.index.get_level_values("cell_num")
cycle_idx = df.index.get_level_values("cycle")
uniq = set(zip(cell_idx, cycle_idx))
uniq

{(26, 21),
 (50, 6),
 (42, 2),
 (28, 67),
 (29, 32),
 (8, 9),
 (19, 18),
 (5, 83),
 (11, 14),
 (50, 91),
 (29, 68),
 (41, 42),
 (33, 38),
 (34, 3),
 (25, 34),
 (39, 81),
 (44, 47),
 (45, 12),
 (10, 54),
 (2, 50),
 (3, 15),
 (22, 28),
 (14, 24),
 (12, 100),
 (13, 65),
 (46, 13),
 (38, 9),
 (24, 74),
 (43, 87),
 (49, 18),
 (35, 83),
 (36, 48),
 (15, 25),
 (1, 90),
 (7, 21),
 (48, 22),
 (5, 60),
 (46, 98),
 (30, 4),
 (9, 67),
 (28, 80),
 (29, 45),
 (41, 19),
 (5, 96),
 (33, 15),
 (25, 11),
 (44, 24),
 (10, 31),
 (2, 27),
 (22, 5),
 (34, 16),
 (20, 81),
 (25, 47),
 (39, 94),
 (12, 77),
 (45, 25),
 (31, 90),
 (23, 86),
 (3, 28),
 (43, 64),
 (14, 37),
 (15, 2),
 (1, 67),
 (47, 34),
 (26, 11),
 (18, 7),
 (35, 96),
 (36, 61),
 (9, 44),
 (28, 57),
 (29, 22),
 (21, 18),
 (5, 73),
 (11, 4),
 (52, 5),
 (9, 80),
 (50, 81),
 (10, 8),
 (8, 84),
 (41, 32),
 (54, 51),
 (33, 28),
 (25, 24),
 (53, 6),
 (39, 71),
 (44, 37),
 (45, 2),
 (31, 67),
 (10, 44),
 (51, 45),
 (2, 40),
 (3, 5),
 (43, 41),
 (22, 18)

In [89]:
from training_utils import _pick_with_nvml, _pick_with_nvidia_smi, pick_idle_gpu

_pick_with_nvidia_smi()

1

In [91]:
_pick_with_nvml()

In [87]:
pick_idle_gpu()

1

In [100]:
from typing import List, Optional, Iterable

a = [1, 2, 3]
b = "a"
isinstance(a, Iterable)

True

In [105]:
tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')

RuntimeError: Visible devices cannot be modified after being initialized

In [99]:
ds.add_C_rate()
df = ds.xCC
test_ids = ['0.8', '0.5']
by='c-rate'
cycles_all = df.index.get_level_values(by).unique()
train = [c for c in cycles_all if c not in test_ids]

mask_tr = df.index.get_level_values(by).isin(train)
mask_te = df.index.get_level_values(by).isin(test_ids)

x_tr = df.loc[mask_tr, :]
x_te = df.loc[mask_te, :]

In [100]:
len(x_tr)/256, len(df)

(20104.03515625, 9785838)

In [95]:
df.index.get_level_values("c-rate").unique()

Index(['0.33', '0.5', '0.8', '1', 'stepped'], dtype='object', name='c-rate')

In [102]:
class Joe:
    def __init__(self, you):
        self.you = you

j = Joe("me")
a = type(j)("me")

In [5]:
from omegaconf import OmegaConf

cfg = OmegaConf.load("/usr/project/chembattery/chembattery_refac/configs/train_lfp_reproduce.yaml")

data_path = os.path.join(cfg.data.data_dir, cfg.data.date_str)
scale_path = os.path.join(cfg.data.data_dir, cfg.data.get("scaler_date", '⚖️'))
features = cfg.data.get("features", None)
dataset = Dataset(data_path, scale_path, features)

trim_to = cfg.data.get("dataset_max", None)
if trim_to is not None:
    dataset.trim(int(trim_to))

prepros_steps = cfg.data.get("preprocessing", [])
print("[reprros: ]", prepros_steps)

print("splitting data")
split_on = cfg.data.get("split_on", "cell_num")
test_split = [str(id) for id in cfg.data.test_split.split(',')]
X_train, X_test, y_train, y_test, scaler = dataset.get_scaled_split(test_split, split_on)

[reprros: ] []
splitting data


In [109]:
X_train, X_test, y_train, y_test = dataset.split(test_split, split_on)

In [115]:
for value in X_train.index.get_level_values("cell_num").unique():
    if value in test_split:
        print(value)

In [111]:
dataset.xCC

Energy   Capacity  Voltage-8    Energy-8   Power-8
cell_num cycle                                                        
1        2       17.887097  11.096774   3.301269   17.758065  0.003499
         2       17.903226  11.096774   3.301331   17.774194  0.003499
         2       17.919355  11.096774   3.301406   17.790323  0.003499
         2       17.935484  11.129032   3.301425   17.806452  0.003500
         2       17.951613  11.129032   3.301505   17.822581  0.003500
...                    ...        ...        ...         ...       ...
55       21     128.387097  74.129032   3.596687  128.177419  0.005719
         21     128.419355  74.161290   3.596916  128.209677  0.005719
         21     128.435484  74.161290   3.597026  128.241935  0.005719
         21     128.467742  74.161290   3.597262  128.258065  0.005720
         21     128.483871  74.193548   3.597437  128.290323  0.005720

[9785838 rows x 5 columns]

In [116]:
model = MLP(cfg)
model.load_model_state("/usr/project/chembattery/chembattery_refac/runs/reproduced_results_1/model/models/model_0")

2025-07-02 20:16:57.223218: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 692 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1a:00.0, compute capability: 8.6
2025-07-02 20:16:57.223763: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22289 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:67:00.0, compute capability: 8.6
2025-07-02 20:16:57.224166: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 21669 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:68:00.0, compute capability: 8.6


In [119]:
X_train, X_test, y_train, y_test, scaler = dataset.get_scaled_split(test_split, split_on)
pred_y = model.predict(X_test)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


   74/67568 [..............................] - ETA: 2:21

2025-07-02 20:17:46.738878: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


67568/67568 [==============================] - 142s 2ms/step


In [120]:
np.sum((pred_y-y_test)**2)**0.5

21902.317066767453

In [13]:
from eval_utils import compute_metrics, print_metrics

In [10]:
preds = []
for i in range(5):
    model = MLP(cfg)
    model.load_model_state(f"/usr/project/chembattery/chembattery_refac/runs/reproduced_results_1/model/models/model_{i}")
    X_train, X_test, y_train, y_test, scaler = dataset.get_scaled_split(test_split, split_on)
    y_pred = model.predict(X_test)
    metrics = compute_metrics(y_pred, y_test)
    print(i)
    print_metrics(metrics)
    print()
    preds.append(y_pred)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


67568/67568 [==============================] - 133s 2ms/step
0

    1/67568 [..............................] - ETA: 1:11:41

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


67568/67568 [==============================] - 132s 2ms/step
1

    1/67568 [..............................] - ETA: 1:13:14

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


67568/67568 [==============================] - 133s 2ms/step
2



/usr/local/lib/python3.8/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


67568/67568 [==============================] - 133s 2ms/step
3

    1/67568 [..............................] - ETA: 1:10:33

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


67568/67568 [==============================] - 133s 2ms/step
4



In [17]:
ensemble_preds = np.array(preds).reshape(5, -1)
ensemble_preds = np.mean(ensemble_preds, axis=0)
metrics = compute_metrics(ensemble_preds, y_test)
print(i)
print(print_metrics(metrics))
print()

4

        "MAE": 0.026895336614543245,
        "Mean Error": -0.011644831910466467,
        "RMSE": 0.03878666672794386,
        "% RMSE": 51.01834028644382,
        "RMSPE": 33716.723830208764,
        "R²": 0.6824957795726763
    



In [15]:
for i,y_pred in enumerate(preds):
    metrics = compute_metrics(y_pred, y_test)
    print(i)
    print(print_metrics(metrics))
    print()

0

        "MAE": 0.031669625880544824,
        "Mean Error": -0.013317887149084884,
        "RMSE": 0.04535128942980448,
        "% RMSE": 59.65316722851638,
        "RMSPE": 43967.90471602293,
        "R²": 0.6428594708234547
    

1

        "MAE": 0.025596248099476877,
        "Mean Error": 0.0027670405088001185,
        "RMSE": 0.037799170309935466,
        "% RMSE": 49.71942928078887,
        "RMSPE": 36666.51100762615,
        "R²": 0.7304664451409433
    

2

        "MAE": 0.04462454734717367,
        "Mean Error": -0.02826921760082949,
        "RMSE": 0.07923569669269127,
        "% RMSE": 104.22328283726358,
        "RMSPE": 44142.33064643632,
        "R²": 0.31143946044485815
    

3

        "MAE": 0.02402684430083716,
        "Mean Error": 0.010083933262275123,
        "RMSE": 0.03474182419074962,
        "% RMSE": 45.69792555694128,
        "RMSPE": 33584.99183263324,
        "R²": 0.7134036763069967
    

4

        "MAE": 0.04245264589170171,
        "Mean Error": -0.0

In [ ]:
np.mean(np.array([
    [1, 2, 3],
    [2, 3, 4]
]), axis=0)

In [123]:
from sklearn.metrics import mean_squared_error

mean_squared_error(pred_y, y_test, squared=False)

61.69414919269864

In [125]:
print(np.sum((pred_y-y_test)**2/len(pred_y))**0.5)

61.69414919269863


In [128]:
s = RobustScaler()
a = np.random.rand(100).reshape(1, -1)
s.fit(a)
b = s.transform(a)
c = s.transform(b)
print(np.max(np.abs(a-b)))

0.9958682421754749


In [ ]:
print(np.max(np.abs(c-b)))

In [129]:
[1]*10

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]